In [1]:
# Clear all current variables
import sys
#sys.modules[__name__].__dict__.clear()

import math
import time
import random
import collections
import numpy as np
from numba import jit
import matplotlib.pyplot as plt


In [2]:
# INIT
N = 3                       # Integer value
n = math.pow(N,3)*4         # Molecule count
n = int(n)
dens = 0.8                  # dens = N / V
V = n / dens                # Volume density
D = math.pow(V,1/3)         # Size of the box
CombC    = 0                # Count for the combinations
MinDist  = D              # This corresponds with Force = 0.05

Kin     = 0.0
Pot     = 0.0
Tot     = 0.0

In [3]:
def InitPos():
    pos = np.zeros((n,3))
    c = 0
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                pos[c,0] = (i+0.5)*D/N; pos[c,1] = (j+0.5)*D/N; pos[c,2] = (k+0.5)*D/N
                pos[c+1,0] = (i+1)*D/N; pos[c+1,1] = (j+0.5)*D/N; pos[c+1,2] = (k+0.5)*D/N
                pos[c+2,0] = (i+0.5)*D/N; pos[c+2,1] = (j+1)*D/N; pos[c+2,2] = (k+0.5)*D/N
                pos[c+3,0] = (i+0.5)*D/N; pos[c+3,1] = (j+0.5)*D/N; pos[c+3,2] = (k+1)*D/N
                c = c + 4
    return pos

def InitVel():
    vel = np.zeros((n,3))
    velmean = np.zeros((3))
    for c in range(0,n):
        for d in range(0,3):
            vel[c][d] = random.uniform(0,1)
            velmean[d] += vel[c][d]
    for c in range(0,n):
        for d in range(0,3):
            vel[c][d] -= velmean[d] / n
    return vel

In [4]:
@jit(nopython=True)
def CalculateDistances(pos):
    CC = 0
    Dis = np.zeros((n**2))
    Cs = np.zeros((n**2))
    
    for i in range(0,n):
        for j in range(i+1,n):
            dis = pos[i,:] - pos[j,:]
            dis = dis - np.rint(dis / (D)) * D
            d = math.sqrt(np.sum(np.power(dis,2)))
            Dis[i*n+j] = d
            #if (d == 0):
             #   print("i = %g\t j = %g"%(i,j))
            if (d < MinDist):
                Cs[CC] = i*n+j
                CC += 1
    return Dis, Cs, CC
                
@jit(nopython=True)
def CalculateForces(pos, CombC, Combs) :
    force = np.zeros((n,3))
    for c in range(0,CombC):
        i = math.floor(Combs[c] / n)
        j = math.floor(Combs[c] % n)
        dis = pos[i,:] - pos[j,:]
        dis = dis - np.rint(dis / D) * D
        d = math.sqrt(np.sum(np.power(dis,2)))
        #dt = d**(-7)
        F = 12 * (2*d**(-13) - d **(-7))
        #F = 12*(2*dt**2*d - dt)
        Fs = F * dis /d
        
        force[i,:] += Fs
        force[j,:] -= Fs
        
    return force

@jit(nopython=True)
def CalulcatePotential(Distance):
    Pot = 0.0
    for i in range(0,n):
        for j in range(i+1,n):
            d = Distance[i*n+j]
            dt = d**(-6)
            Pot += dt**2 - 2*dt
    return Pot

pos = InitPos()
vel = InitVel()
start_time = time.time()
Distance, Combs, CombC = CalculateDistances(pos)
print(time.time()-start_time)
start_time = time.time()
force = CalculateForces(pos, CombC, Combs)
print(time.time()-start_time)
start_time = time.time()
pot = CalulcatePotential(Distance)
print(time.time()-start_time)

0.5093402862548828
0.5283536911010742
0.14609837532043457


In [ ]:
#@jit
def Calculate():
    KinArr = []
    PotArr = []
    Time = []

    pos = InitPos()
    vel = InitVel()
    
    Distance, Combs, CombC = CalculateDistances(pos)
    force = CalculateForces(pos, CombC, Combs)
    veltemp = vel
    t_last = 0
    t = 0
    while t <= t_end:
        if (t >= t_last):
            Distance, Combs, CombC = CalculateDistances(pos)
            Pot = CalulcatePotential(Distance)
            # Calculate the Kinetic energy
            Kin = 0
            for c in range(0,n):
                Kin += math.sqrt(vel[c][0]**2+vel[c][1]**2+vel[c][2]**2)
            KinArr.append(Kin)
            PotArr.append(Pot)
            Time.append(t)
            #print("Time = %.3g"%(t))
            sys.stdout.write("\r%2.3g%%\t\t%g" % (math.floor(t/t_end * 1000)/10, CombC))
            sys.stdout.flush()
            t_last += 0.005
        
        vel += force*(dt/2)
        pos += vel*dt
        pos = pos%D
        force = CalculateForces(pos, CombC, Combs)
        vel += force*(dt/2)
        t += dt
    return Time, KinArr, PotArr
    
dt = 4E-3
ts = 5000
t_end = dt * ts
start_time = time.time()
Time, KinArr, PotArr = Calculate()
time_spend = time.time() - start_time
time_it = time_spend/ts
print("\nTime = %.6f"%(time_it))

99.9%		5778
Time = 0.008189


In [ ]:
TotArr = []
c = len(PotArr)
for i in range(0, c):
    TotArr.append(KinArr[i] + PotArr[i])
plt.plot(Time, PotArr)
plt.plot(Time, KinArr)
plt.plot(Time, TotArr)
#plt.plot(Time, PotArr + KinArr)
plt.legend(("Potential Energy", "Kinetic Energy", "Total Energy"))
plt.title("Energy in %g argo atoms"%(n))
plt.show()